<a href="https://colab.research.google.com/github/anjelammcgraw/Generative-Supervised-Fine-tuning-of-GPT-2/blob/main/3_Generative_Supervised_Fine_tuning_of_GPT_2_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Generative Supervised Fine-tuning of GPT-2



##Dependencies

In [ ]:
!pip install transformers accelerate datasets trl bitsandbytes -qU

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.6/79.6 kB 6.0 MB/s eta 0:00:00


## Dataset Curation

We're going to be fine-tuning our model on SQL generation t

We'll use [this](https://huggingface.co/datasets/b-mc2/sql-create-context) dataset.


In [ ]:
from datasets import load_dataset

sql_dataset = load_dataset("b-mc2/sql-create-context")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
sql_dataset

DatasetDict({
    train: Dataset({
        features: ['question', 'context', 'answer'],
        num_rows: 78577
    })
})

In [ ]:
sql_dataset['train'][0]

{'question': 'How many heads of the departments are older than 56 ?',
 'context': 'CREATE TABLE head (age INTEGER)',
 'answer': 'SELECT COUNT(*) FROM head WHERE age > 56'}

In [ ]:
sql_dataset_train_test = sql_dataset["train"].train_test_split(test_size=0.2)

In [ ]:
sql_dataset_train_test

DatasetDict({
    train: Dataset({
        features: ['question', 'context', 'answer'],
        num_rows: 62861
    })
    test: Dataset({
        features: ['question', 'context', 'answer'],
        num_rows: 15716
    })
})

In [ ]:
sql_dataset_val_test = sql_dataset_train_test["test"].train_test_split(test_size=0.5)

In [ ]:
sql_dataset_val_test

DatasetDict({
    train: Dataset({
        features: ['question', 'context', 'answer'],
        num_rows: 7858
    })
    test: Dataset({
        features: ['question', 'context', 'answer'],
        num_rows: 7858
    })
})

In [ ]:
from datasets import DatasetDict

split_sql_dataset = DatasetDict({
    "train" : sql_dataset_train_test["train"],
    "val" : sql_dataset_val_test["train"],
    "test" : sql_dataset_val_test["test"]
})

In [ ]:
split_sql_dataset

DatasetDict({
    train: Dataset({
        features: ['question', 'context', 'answer'],
        num_rows: 62861
    })
    val: Dataset({
        features: ['question', 'context', 'answer'],
        num_rows: 7858
    })
    test: Dataset({
        features: ['question', 'context', 'answer'],
        num_rows: 7858
    })
})

##Create a Prompt

In [ ]:
TEXT2SQL_TRAINING_PROMPT_TEMPLATE = """
[ ]
You are a powerful text-to-SQL model. Your job is to answer questions about a database. You are given a question and context regarding one or more tables.
You have the following input:
Question: "{question}"
Context: "{context}"
Tables: {table_list}
Please generate the SQL query that answers the question.
"""

TEXT2SQL_INFERENCE_PROMPT_TEMPLATE = """
[ ]
You are a powerful text-to-SQL model. Your job is to answer questions about a database. You are given a question and context regarding one or more tables.
Please generate the SQL query that answers the question.
"""

In [ ]:
SYSTEM_MESSAGE = "You are a powerful text-to-SQL model. Your job is to answer questions about a database. You are given a question and context regarding one or more tables. Please generate the SQL query that answers the question."

In [ ]:
def create_sql_prompt(sample):
  full_prompt = TEXT2SQL_TRAINING_PROMPT_TEMPLATE.format(
      bos_token = "<|startoftext|>",
      eos_token = "<|endoftext|>",
      system_message = SYSTEM_MESSAGE,
      question = sample["question"],
      context = sample["context"],
      table_list = sample["answer"]
  )

  return {"text" : full_prompt}

In [ ]:
def create_sql_prompt_and_response(sample):
  full_prompt = TEXT2SQL_INFERENCE_PROMPT_TEMPLATE.format(
      bos_token = "<|startoftext|>",
      eos_token = "<|endoftext|>",
      system_message = SYSTEM_MESSAGE,
      input = sample["question"],
      context = sample["context"],
      response = sample["answer"]

  )

  ground_truth = sample["answer"]

  return {"full_prompt" : full_prompt, "ground_truth" : ground_truth}

In [ ]:
create_sql_prompt(split_sql_dataset["train"][0])

{'text': '\n[ ]\nYou are a powerful text-to-SQL model. Your job is to answer questions about a database. You are given a question and context regarding one or more tables.\nYou have the following input:\nQuestion: "Which team has the team captain Jon Newsome?"\nContext: "CREATE TABLE table_name_81 (team VARCHAR, captain VARCHAR)"\nTables: SELECT team FROM table_name_81 WHERE captain = "jon newsome"\nPlease generate the SQL query that answers the question.\n'}

In [ ]:
split_sql_dataset = split_sql_dataset.map(create_sql_prompt)

Map:   0%|          | 0/62861 [00:00<?, ? examples/s]

Map:   0%|          | 0/7858 [00:00<?, ? examples/s]

Map:   0%|          | 0/7858 [00:00<?, ? examples/s]

## Load the Model And Preproccessing

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_id = "gpt2"

gpt2_base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto"
)

gpt2_tokenizer = AutoTokenizer.from_pretrained(model_id)

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
gpt2_tokenizer.pad_token = gpt2_tokenizer.eos_token

In [ ]:
gpt2_base_model.resize_token_embeddings(len(gpt2_tokenizer))

Embedding(50257, 768)

In [ ]:
from transformers import pipeline, set_seed, GenerationConfig
generator = pipeline('text-generation', model=gpt2_base_model, tokenizer=gpt2_tokenizer)
set_seed(42)

def generate_sample(sample):
  prompt_package = create_sql_prompt_and_response(sample)

  generation_config = GenerationConfig(
      max_new_tokens=50,
      do_sample=True,
      top_k=50,
      temperature=1e-4,
      eos_token_id=gpt2_base_model.config.eos_token_id,
  )

  generation = generator(prompt_package["full_prompt"], generation_config=generation_config)
  print("---------------")
  print("Model Response:")
  print(generation[0]["generated_text"].replace(prompt_package["full_prompt"], ""))
  print("+++++++++++++++")
  print("Ground Truth")
  print(prompt_package["ground_truth"])

In [ ]:
generate_sample(split_sql_dataset["test"][0])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


---------------
Model Response:
You are given a question and context regarding one or more tables. You are given a question and context regarding one or more tables. You are given a question and context regarding one or more tables. You are given a question and context regarding one or more
+++++++++++++++
Ground Truth
SELECT game FROM table_name_33 WHERE date = "april 28"


## Training the Model

In [ ]:
from transformers import TrainingArguments
from trl import SFTTrainer

training_args = TrainingArguments(
 per_device_train_batch_size=4,
 gradient_accumulation_steps=4,
 gradient_checkpointing =True,
 max_grad_norm= 0.3,
 ###num_train_epochs=2,
 max_steps=500,
 learning_rate=2e-4,
 save_total_limit=3,
 logging_steps=10,
 output_dir="sql_gpt2",
 optim="paged_adamw_32bit",
 lr_scheduler_type="cosine",
 evaluation_strategy="steps",
 eval_steps=50,
 warmup_ratio=0.05,
)

###❓Question

Is this process using usupervised, or supervised learning?

**ANSWER** This process is using supervised learning.

In [ ]:
trainer = SFTTrainer(
 gpt2_base_model,
 dataset_text_field="text",
 train_dataset=split_sql_dataset["train"],
 eval_dataset=split_sql_dataset["val"],
 tokenizer=gpt2_tokenizer,
 max_seq_length=1024,
 args=training_args
)

Map:   0%|          | 0/62861 [00:00<?, ? examples/s]

Map:   0%|          | 0/7858 [00:00<?, ? examples/s]

###❓Question

What do we use to determine loss in fine-tuning GPT-2?

> HINT: [This](https://github.com/huggingface/transformers/blob/main/src/transformers/models/gpt2/modeling_gpt2.py) can help you if you get stuck!

**Answer** In fine-tuning GPT-2, we use CrossEntropyLoss (a language modeling objective).

In [ ]:
trainer.train()

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss
50,0.749100,0.708043
100,0.708500,0.654632
150,0.663300,0.626485
200,0.632800,0.605830
250,0.655300,0.590459
300,0.615000,0.580634
350,0.575800,0.570206
400,0.591300,0.564149
450,0.589800,0.561876
500,0.573500,0.561403


TrainOutput(global_step=500, training_loss=0.6932152862548828, metrics={'train_runtime': 1707.7589, 'train_samples_per_second': 4.685, 'train_steps_per_second': 0.293, 'total_flos': 616716559872000.0, 'train_loss': 0.6932152862548828, 'epoch': 0.13})

In [ ]:
trainer.save_model()

## Testing our Model

In [ ]:
ft_gpt2_model = AutoModelForCausalLM.from_pretrained("sql_gpt2")

In [ ]:
generator = pipeline('text-generation', model=ft_gpt2_model, tokenizer=gpt2_tokenizer, )

In [ ]:
generate_sample(split_sql_dataset["test"][1])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


---------------
Model Response:
You have the following input:
Question: "What is the average attendance for the game on March 22?"
Context: "CREATE TABLE table_name_87 (attendance INTEGER, date VARCHAR)"
Tables
+++++++++++++++
Ground Truth
SELECT COUNT(rank) FROM table_name_63 WHERE bronze < 0


That is *significantly* better.

####❓Question

How might you evaluate your generated SQL? Please provide 3 different methods.

**ANSWER** We can evaluate the generated SQL model with three methods:

1. Using a SQL parser or query execution engine
2. Executing the generated SQL queries on a real dataset (compare results with expected output)
3. Utilizing domain experts/SQL professionals review the generated SQL queries

However, a multifaceted approach instead of a single method is sufficient to ensure reliability/quality.